# LoRA Training Notebook

**Phan Đức Thành Phát (23521149)**

This notebook will cover LoRA fine-tuning for selected WikiArt styles.

## TODO

- [ ] Mount datasets and verify captions
- [ ] Configure LoRA hyperparameters (rank, learning rate, etc.)
- [ ] Setup LoRA layers on UNet attention modules
- [ ] Launch training with accelerate
- [ ] Export LoRA checkpoints and sample outputs
